<a href="https://colab.research.google.com/github/Dark-Sied/Intent_Classification/blob/master/Intent_classification_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intent Recognition
Article: https://towardsdatascience.com/a-brief-introduction-to-intent-classification-96fda6b1f557    
Changed by Christoph Windheuser, April 2020

In [1]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.lancaster import LancasterStemmer
import nltk
import re
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.models import Sequential, load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint


Using TensorFlow backend.


In [6]:
df = pd.read_csv("Dataset.csv", encoding = "latin1", names = ["Sentence", "Intent"])
print(df.head())
intent = df["Intent"]
unique_intent = list(set(intent))
sentences = list(df["Sentence"])


                Sentence          Intent
0       Need help pleese  commonQ.assist
1              Need help  commonQ.assist
2       I need some info  commonQ.assist
3      Will you help me?  commonQ.assist
4  What else can you do?  commonQ.assist


In [7]:
#intent, unique_intent, sentences = load_dataset("Dataset.csv")

In [8]:
df.head()

,Sentence,Intent
0,Need help pleese,commonQ.assist
1,Need help,commonQ.assist
2,I need some info,commonQ.assist
3,Will you help me?,commonQ.assist
4,What else can you do?,commonQ.assist


In [9]:
df.shape

(1113, 2)

In [11]:
print ("No of intents: ", len(unique_intent))
print (unique_intent)


No of intents:  21
['commonQ.wait', 'commonQ.just_details', 'commonQ.not_giving', 'commonQ.name', 'faq.borrow_use', 'commonQ.query', 'commonQ.bot', 'contact.contact', 'faq.borrow_limit', 'faq.address_proof', 'commonQ.assist', 'faq.approval_time', 'faq.biz_category_missing', 'faq.banking_option_missing', 'faq.apply_register', 'faq.aadhaar_missing', 'faq.bad_service', 'faq.application_process', 'faq.biz_new', 'faq.biz_simpler', 'commonQ.how']


In [12]:
print(sentences[:5])

['Need help pleese', 'Need help', 'I need some info', 'Will you help me?', 'What else can you do?']


In [13]:
nltk.download("stopwords")
nltk.download("punkt")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cwindheu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /Users/cwindheu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [14]:
#define stemmer
stemmer = LancasterStemmer()

In [15]:
def cleaning(sentences):
  words = []
  for s in sentences:
    clean = re.sub(r'[^ a-z A-Z 0-9]', " ", s)
    w = word_tokenize(clean)
    #stemming
    words.append([i.lower() for i in w])
    
  return words  

In [16]:
cleaned_words = cleaning(sentences)
print(len(cleaned_words))
print(cleaned_words[:2])  
  


1113
[['need', 'help', 'pleese'], ['need', 'help']]


In [17]:
def create_tokenizer(words, filters = '!"#$%&()*+,-./:;<=>?@[\]^_`{|}~'):
  token = Tokenizer(filters = filters)
  token.fit_on_texts(words)
  return token

In [18]:
def max_length(words):
  return(len(max(words, key = len)))
  

In [19]:
word_tokenizer = create_tokenizer(cleaned_words)
vocab_size = len(word_tokenizer.word_index) + 1
max_length = max_length(cleaned_words)

print("Vocab Size = %d and Maximum length = %d" % (vocab_size, max_length))

Vocab Size = 492 and Maximum length = 28


In [20]:
def encoding_doc(token, words):
  return(token.texts_to_sequences(words))

In [21]:
encoded_doc = encoding_doc(word_tokenizer, cleaned_words)

In [22]:
def padding_doc(encoded_doc, max_length):
  return(pad_sequences(encoded_doc, maxlen = max_length, padding = "post"))

In [23]:
padded_doc = padding_doc(encoded_doc, max_length)

In [24]:
padded_doc[:5]

array([[ 25,  77, 332,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [ 25,  77,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  1,  25, 198, 181,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [ 51,  10,  77,  16,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0],
       [  8, 268,   4,  10,  30,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0]], dtype=int32)

In [25]:
print("Shape of padded docs = ",padded_doc.shape)

Shape of padded docs =  (1113, 28)


In [26]:
#tokenizer with filter changed
output_tokenizer = create_tokenizer(unique_intent, filters = '!"#$%&()*+,-/:;<=>?@[\]^`{|}~')


In [27]:
output_tokenizer.word_index

{'commonq.wait': 1,
 'commonq.just_details': 2,
 'commonq.not_giving': 3,
 'commonq.name': 4,
 'faq.borrow_use': 5,
 'commonq.query': 6,
 'commonq.bot': 7,
 'contact.contact': 8,
 'faq.borrow_limit': 9,
 'faq.address_proof': 10,
 'commonq.assist': 11,
 'faq.approval_time': 12,
 'faq.biz_category_missing': 13,
 'faq.banking_option_missing': 14,
 'faq.apply_register': 15,
 'faq.aadhaar_missing': 16,
 'faq.bad_service': 17,
 'faq.application_process': 18,
 'faq.biz_new': 19,
 'faq.biz_simpler': 20,
 'commonq.how': 21}

In [28]:
encoded_output = encoding_doc(output_tokenizer, intent)

In [29]:
encoded_output = np.array(encoded_output).reshape(len(encoded_output), 1)

In [30]:
encoded_output.shape

(1113, 1)

In [31]:
def one_hot(encode):
  o = OneHotEncoder(sparse = False)
  return(o.fit_transform(encode))

In [32]:
output_one_hot = one_hot(encoded_output)

In [33]:
output_one_hot.shape

(1113, 21)

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
train_X, val_X, train_Y, val_Y = train_test_split(padded_doc, output_one_hot, shuffle = True, test_size = 0.2)


In [36]:
print("Shape of train_X = %s and train_Y = %s" % (train_X.shape, train_Y.shape))
print("Shape of val_X = %s and val_Y = %s" % (val_X.shape, val_Y.shape))

Shape of train_X = (890, 28) and train_Y = (890, 21)
Shape of val_X = (223, 28) and val_Y = (223, 21)


In [37]:
def create_model(vocab_size, max_length):
  model = Sequential()
  model.add(Embedding(vocab_size, 128, input_length = max_length, trainable = False))
  model.add(Bidirectional(LSTM(128)))
#   model.add(LSTM(128))
  model.add(Dense(32, activation = "relu"))
  model.add(Dropout(0.5))
  model.add(Dense(21, activation = "softmax"))
  
  return model

In [38]:
model = create_model(vocab_size, max_length)

model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 28, 128)           62976     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 256)               263168    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                8224      
_________________________________________________________________
dropout_1 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 21)                693       
Total params: 335,061
Trainable params: 272,085
Non-trainable par

In [39]:
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

hist = model.fit(train_X, train_Y, epochs = 100, batch_size = 32, validation_data = (val_X, val_Y), callbacks = [checkpoint])


Instructions for updating:
Use tf.cast instead.
Train on 890 samples, validate on 223 samples
Epoch 1/100
890/890 [==============================] - 2s 3ms/step - loss: 2.9474 - acc: 0.1708 - val_loss: 2.7870 - val_acc: 0.1794

Epoch 00001: val_loss improved from inf to 2.78701, saving model to model.h5
Epoch 2/100
890/890 [==============================] - 1s 1ms/step - loss: 2.8336 - acc: 0.1674 - val_loss: 2.7424 - val_acc: 0.1794

Epoch 00002: val_loss improved from 2.78701 to 2.74238, saving model to model.h5
Epoch 3/100
890/890 [==============================] - 1s 1ms/step - loss: 2.7862 - acc: 0.1607 - val_loss: 2.6757 - val_acc: 0.1794

Epoch 00003: val_loss improved from 2.74238 to 2.67568, saving model to model.h5
Epoch 4/100
890/890 [==============================] - 1s 1ms/step - loss: 2.7413 - acc: 0.1730 - val_loss: 2.6069 - val_acc: 0.1794

Epoch 00004: val_loss improved from 2.67568 to 2.60689, saving model to model.h5
Epoch 5/100
890/890 [=============================

890/890 [==============================] - 1s 1ms/step - loss: 0.9320 - acc: 0.7124 - val_loss: 1.1737 - val_acc: 0.6996

Epoch 00039: val_loss improved from 1.17524 to 1.17370, saving model to model.h5
Epoch 40/100
890/890 [==============================] - 1s 1ms/step - loss: 0.9072 - acc: 0.7101 - val_loss: 1.1283 - val_acc: 0.7175

Epoch 00040: val_loss improved from 1.17370 to 1.12828, saving model to model.h5
Epoch 41/100
890/890 [==============================] - 1s 1ms/step - loss: 0.8516 - acc: 0.7281 - val_loss: 1.0959 - val_acc: 0.7130

Epoch 00041: val_loss improved from 1.12828 to 1.09588, saving model to model.h5
Epoch 42/100
890/890 [==============================] - 1s 1ms/step - loss: 0.8470 - acc: 0.7236 - val_loss: 1.0947 - val_acc: 0.6816

Epoch 00042: val_loss improved from 1.09588 to 1.09468, saving model to model.h5
Epoch 43/100
890/890 [==============================] - 1s 1ms/step - loss: 0.8334 - acc: 0.7416 - val_loss: 1.3204 - val_acc: 0.6816

Epoch 00043: v


Epoch 00081: val_loss did not improve from 0.94471
Epoch 82/100
890/890 [==============================] - 1s 1ms/step - loss: 0.4604 - acc: 0.8461 - val_loss: 0.9999 - val_acc: 0.7668

Epoch 00082: val_loss did not improve from 0.94471
Epoch 83/100
890/890 [==============================] - 1s 1ms/step - loss: 0.4500 - acc: 0.8584 - val_loss: 1.1473 - val_acc: 0.7399

Epoch 00083: val_loss did not improve from 0.94471
Epoch 84/100
890/890 [==============================] - 1s 1ms/step - loss: 0.4470 - acc: 0.8551 - val_loss: 1.2442 - val_acc: 0.7309

Epoch 00084: val_loss did not improve from 0.94471
Epoch 85/100
890/890 [==============================] - 1s 1ms/step - loss: 0.7944 - acc: 0.7674 - val_loss: 1.0499 - val_acc: 0.7265

Epoch 00085: val_loss did not improve from 0.94471
Epoch 86/100
890/890 [==============================] - 1s 1ms/step - loss: 0.7429 - acc: 0.7865 - val_loss: 0.9816 - val_acc: 0.7578

Epoch 00086: val_loss did not improve from 0.94471
Epoch 87/100
890/8

In [40]:
 model = load_model("model.h5")

In [41]:
def predictions(text):
  clean = re.sub(r'[^ a-z A-Z 0-9]', " ", text)
  test_word = word_tokenize(clean)
  test_word = [w.lower() for w in test_word]
  test_ls = word_tokenizer.texts_to_sequences(test_word)
  print(test_word)
  #Check for unknown words
  if [] in test_ls:
    test_ls = list(filter(None, test_ls))
    
  test_ls = np.array(test_ls).reshape(1, len(test_ls))
 
  x = padding_doc(test_ls, max_length)
  
  pred = model.predict_proba(x)
  
  
  return pred


  

In [42]:
def get_final_output(pred, classes):
  predictions = pred[0]
 
  classes = np.array(classes)
  ids = np.argsort(-predictions)
  classes = classes[ids]
  predictions = -np.sort(-predictions)
 
  for i in range(pred.shape[1]):
    print("%s has confidence = %s" % (classes[i], (predictions[i])))



In [45]:
text = "Can you help me with registration?"
pred = predictions(text)
get_final_output(pred, unique_intent)

['can', 'you', 'help', 'me', 'with', 'registration']
commonQ.name has confidence = 0.22885664
commonQ.assist has confidence = 0.1676464
contact.contact has confidence = 0.16310427
commonQ.bot has confidence = 0.16069105
commonQ.how has confidence = 0.08810879
faq.apply_register has confidence = 0.069628015
commonQ.query has confidence = 0.049808335
faq.bad_service has confidence = 0.024044132
commonQ.not_giving has confidence = 0.023743084
commonQ.wait has confidence = 0.016602974
faq.biz_new has confidence = 0.0029173347
commonQ.just_details has confidence = 0.0027844
faq.borrow_use has confidence = 0.0010256921
faq.application_process has confidence = 0.00058372103
faq.aadhaar_missing has confidence = 0.00022147712
faq.biz_simpler has confidence = 7.344989e-05
faq.address_proof has confidence = 7.184449e-05
faq.banking_option_missing has confidence = 4.4712117e-05
faq.borrow_limit has confidence = 2.8277327e-05
faq.biz_category_missing has confidence = 1.4730535e-05
faq.approval_time